In [1]:
from enum import Enum
import numpy as np
# from usefulFunctions import *
import copy
import  pickle
import json

In [2]:
class Status(Enum):
    STAGED = 200
    ENQUEUED = 201
    DONE = 202

In [3]:
class Task:
    # __actionDict__ = {0:np.array([0,0]), 1:np.array([0,1]), 2:np.array([1,0]), 3:np.array([0,-1]), 4:np.array([-1,0])}
    __actionDict__ = {0:np.array([0,0]), 1:np.array([1,0]), 2:np.array([0,1]), 3:np.array([-1,0]), 4:np.array([0,-1])}

    def __init__(self, tid, rid, start, action, time) -> None:
        self.taskID = tid
        self.robotID = rid
        self.action = action
        self.startPos = np.array(start)
        self.goalPos = np.array(self.__actionDict__[action]+start)
        self.time = time
        self.preqs = dict()
        self.status = Status.STAGED
        
    def __checkIfReady__(self):
        # print("Check if ready", self.preqs)
        if self.taskID-1 not in self.preqs:
            temp = list(self.preqs.values())
            if(np.array_equal(temp, np.full_like(temp, Status.DONE.value))):
                return True
            else:
                return False
        if self.preqs[self.taskID-1]>=Status.ENQUEUED.value:
            temp = copy.deepcopy(self.preqs)
            del temp[self.taskID-1]
            temp = list(temp.values())
            if (np.array_equal(temp, np.full_like(temp, Status.DONE.value))):
                return True
            else:
                return False
        return False
    
    def __publishTask__(self):
        global TOPIC
        message = ROS_Task()
        message.taskID = self.taskID
        message.robotID = self.robotID
        message.action = self.action
        message.status = self.status.value

        TOPIC.publish(message)

    
    def __repr__(self) -> str:
        asd = "\n"
        for i in self.__dir__():
            if not i.startswith('__'):
                asd+=i
                asd+=": "
                asd+=str(getattr(self,i))
                asd+=", "

        return asd

class Graph:
    def __init__(self, taskList, startCells) -> None:
        self.currentPositions = copy.deepcopy(startCells)
        self.graph = dict()
        self.taskDict = dict()
        tId = 1
        self.robotList = []
        for rid in range(len(startCells)):
        
            prevTask = None
            for i, task in enumerate(taskList[:,rid]):
                t = Task(tId, rid, self.currentPositions[rid], task, i)
                # print(t)
                self.taskDict[tId] = t
                tId+=1
                self.currentPositions[rid] = t.goalPos
                self.addVertex(t)

                if(prevTask is None):
                    self.robotList.append(t)
                else:
                    self.addEdge(prevTask, t)

                prevTask = t
        # print(taskDict)
        for rid in range(len(startCells)):
            firstTid = self.robotList[rid].taskID
            for taskID in range(firstTid, firstTid+len(taskList)):
                if(taskID not in self.taskDict):
                    continue
                task = self.taskDict[taskID]
                
                for rid_ in range(len(startCells)):
                    if(rid != rid_):
                        # print(rid, rid_)
                        firstTid_ = self.robotList[rid_].taskID
                        for taskID_ in range(firstTid_, firstTid_+len(taskList)):
                            if(taskID_ not in self.taskDict):
                                continue
                            task_ = self.taskDict[taskID_]
                            if np.array_equal(task.startPos, task_.goalPos) and task.time<=task_.time:
                                # print(task, task_)
                                self.addEdge(task, task_)
                                break

            
    
    def addVertex(self, task):
        assert type(task) == Task
        if(task.taskID not in self.graph):
            self.graph[task.taskID] = list()

    def addEdge(self, taskFrom, taskTo):
        assert  type(taskFrom)==Task and taskFrom.taskID in self.graph
        assert type(taskTo)==Task and taskTo.taskID in self.graph
        taskTo.preqs[taskFrom.taskID] = taskFrom.status.value
        self.graph[taskFrom.taskID].append(taskTo.taskID)

    def changeStatus(self, taskID):
        # rospy.loginfo(taskID)
        task = self.taskDict[taskID]
        # print(taskID, task)
        task.status = Status(task.status.value+1)

        task.__publishTask__()

In [4]:
actionList= np.loadtxt('actions.txt')
actionList = actionList.astype(np.int16)

size = actionList.shape

START=[(2, 7),(20, 1),(17, 16),(2, 0),(10, 1),(16, 22),(17, 7),(1, 4),(5, 5),(17, 18),(7, 22),(21, 20),(7, 1),(1, 2),(15, 21),(2, 17),(5, 10),(17, 13),(10, 22),(2, 21),(8, 4),(8, 19),(5, 3),(20, 3),(14, 12),(19, 20),(1, 7),(2, 14),(14, 21),(15, 2),(11, 21),(17, 1),(2, 10),(5, 9),(1, 5),(13, 1),(22, 2),(15, 14),(3, 21),(2, 8),(1, 0),(22, 16),(0, 14),(21, 2),(8, 22),(18, 8),(18, 1),(0, 20),(1, 22),(17, 5),(11, 8),(17, 11),(8, 17),(0, 9),(16, 0),(17, 2),(4, 8),(14, 19),(2, 11),(12, 14),(8, 6),(21, 1),(2, 16),(21, 17),(0, 18),(19, 22),(20, 16),(9, 2),(0, 17),(7, 8),(1, 6),(11, 0),(17, 3),(0, 0),(9, 14),(22, 10),(8, 9),(21, 16),(21, 15),(1, 1),(20, 15),(19, 1),(9, 22),(22, 3),(2, 9),(7, 21),(14, 13),(0, 21),(12, 8),(0, 2),(18, 20),(17, 22),(1, 10),(0, 10),(5, 4),(22, 5),(13, 8),(2, 22),(5, 15),(6, 14),(1, 13),(5, 19),(22, 7),(0, 12),(15, 22),(21, 19),(5, 20),(10, 8),(21, 7),(22, 0),(7, 2),(1, 16),(11, 19),(14, 8),(8, 5),(1, 19),(1, 11),(0, 5),(1, 20),(8, 7),(5, 18),(14, 6),(22, 21),(14, 17),(6, 2),(18, 14),(15, 20),(12, 22),(21, 12),(6, 8),(0, 7),(11, 17),(22, 20),(0, 11),(11, 6),(14, 2),(2, 3),(16, 2),(14, 10),(0, 13),(16, 1),(15, 1),(21, 21),(18, 21),(0, 4),(5, 22),(20, 6),(1, 9),(20, 14),(12, 0),(0, 8),(20, 5),(20, 13),(12, 2),(13, 22),(1, 3),(21, 10),(11, 3),(4, 1),(17, 21),(20, 18),(14, 3),(14, 14),(0, 6),(7, 14),(22, 8),(8, 1),(2, 19),(12, 1),(20, 4),(18, 0),(1, 17),(5, 1),(22, 18),(19, 0),(22, 9),(9, 8),(17, 17),(2, 12),(21, 8),(8, 21),(20, 19),(5, 11),(17, 10),(17, 6),(22, 14),(3, 20),(11, 15),(3, 2),(8, 3),(17, 15),(13, 21),(18, 22),(14, 1),(21, 3),(6, 0),(8, 8),(14, 11),(21, 13),(11, 1),(10, 20),(22, 13),(5, 13),(6, 21),(0, 22),(3, 22),(0, 16),(3, 14),(4, 14),(1, 15),(5, 14),(8, 15)]

REAL_ROBOT_COUNT = 10
VIRTUAL_ROBOT_COUNT = 30

REAL_ROBOT_START = START[202:]
START = START[:VIRTUAL_ROBOT_COUNT]
START+=REAL_ROBOT_START[:REAL_ROBOT_COUNT]


taskList = np.zeros((size[0], REAL_ROBOT_COUNT+VIRTUAL_ROBOT_COUNT),dtype=np.int16)

for i in range(REAL_ROBOT_COUNT+VIRTUAL_ROBOT_COUNT):
    if i>=VIRTUAL_ROBOT_COUNT:
        taskList[:,i] = actionList[:,202+i-VIRTUAL_ROBOT_COUNT]
    else:
        taskList[:,i] = actionList[:,i]

In [5]:
GRAPH = Graph(actionList, START)

temp = {}
for i in GRAPH.graph:
    temp[i] = {}
    temp[i]["action"] = int(GRAPH.taskDict[i].action)
    temp[i]["preqs"] = list(GRAPH.taskDict[i].preqs.keys())
    temp[i]["robotID"] = GRAPH.taskDict[i].robotID

with open("/home/mapf/td_ws/src/Tanishq_MAPF/database/tasklist-30-10.txt", "w") as f:
    f.write(json.dumps(temp))


with open("/home/mapf/td_ws/src/Tanishq_MAPF/database/graph-30-10.txt", "w") as f:
    f.write(json.dumps(GRAPH.graph))

In [6]:
# with open("/home/mapf/td_ws/src/Tanishq_MAPF/database/tasklist-100-10.txt") as f:
#     t = json.load(f)

# with open("/home/mapf/td_ws/src/Tanishq_MAPF/database/graph-100-10.txt") as f:
#     g = json.load(f)

In [9]:
# G2 = Graph([],[])

In [19]:
# for i in g:
#     taskID = int(i)
#     G2.graph[taskID] = g[i]
#     t_ = Task(taskID, t[str(taskID)]['robotID'],[-1,-1],t[str(taskID)]['action'],-1)
#     for pre in t[str(taskID)]['preqs']:
#         t_.preqs[pre] = Status.STAGED
#     G2.taskDict[taskID] = t_

In [20]:
# G2.taskDict

{1: 
 taskID: 1, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {}, status: Status.STAGED, ,
 2: 
 taskID: 2, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {1: <Status.STAGED: 200>}, status: Status.STAGED, ,
 3: 
 taskID: 3, robotID: 0, action: 3, startPos: [-1 -1], goalPos: [-2 -1], time: -1, preqs: {2: <Status.STAGED: 200>, 1613: <Status.STAGED: 200>}, status: Status.STAGED, ,
 4: 
 taskID: 4, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {3: <Status.STAGED: 200>}, status: Status.STAGED, ,
 5: 
 taskID: 5, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {4: <Status.STAGED: 200>}, status: Status.STAGED, ,
 6: 
 taskID: 6, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {5: <Status.STAGED: 200>}, status: Status.STAGED, ,
 7: 
 taskID: 7, robotID: 0, action: 0, startPos: [-1 -1], goalPos: [-1 -1], time: -1, preqs: {6: <Status.STAGED: 200>}